In [1]:
import os
import numpy as np
import pandas as pd
from math import ceil
from collections import deque
import math
import sys
global maxdays
current_directory = os.getcwd()
print("Current working directory:", current_directory)


Current working directory: /Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project


In [2]:
%cd /Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project


/Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project


In [3]:
"""
Enter filepath of instance and desired name for the solution file
"""
directory = "/Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project"
filename = 'CO_Case2405.txt'
output_file_name = "CO_Case2405_sol.txt"


"""
 Read the instance and assign variables to the different values
"""

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}

print("Machines:")
print(machines)

print("\nLocations:")
print(locations)

print("\nRequests:")
print(requests)

print("\nTechnicians:")
print(technicians)


print(DAYS)
print(TRUCK_CAPACITY)
print(TRUCK_MAX_DISTANCE)

print(TRUCK_DISTANCE_COST)
print(TRUCK_DAY_COST)
print(TRUCK_COST)
print(TECHNICIAN_DISTANCE_COST)
print(TECHNICIAN_DAY_COST)
print(TECHNICIAN_COST)

Machines:
[[  1   3 345]
 [  2   7 995]]

Locations:
[[  1 575 363]
 [  2 649 337]
 [  3 462 402]
 [  4 339 513]
 [  5 365 182]
 [  6 490 572]
 [  7 559 411]
 [  8 735 589]
 [  9 582 711]]

Requests:
[[ 1  6  2  3  1  3]
 [ 2  8  3  4  1  1]
 [ 3  8  4  7  2  1]
 [ 4  3  4  5  1  3]
 [ 5  7  4  7  2  2]
 [ 6  6  4  7  2  2]
 [ 7  8  4  6  1  2]
 [ 8  6  5  7  1  2]
 [ 9  6  5  7  2  2]
 [10  6  6  7  2  2]]

Technicians:
[[  1   9 610   8   1   1]
 [  2   1 270   5   1   1]
 [  3   4 455   7   1   1]
 [  4   1 395   8   0   1]
 [  5   1 470   8   1   1]]
10
15
555
0
0
0
10
0
0


In [4]:
def nearest_neighbor(point, unvisited_points, locations):
    """
    Find the nearest neighbor of a given point among a list of unvisited points.

    Args:
    - point: The point for which the nearest neighbor is to be found.
    - unvisited_points: List of unvisited points.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - nearest_point: The nearest neighbor of the given point.
    """
    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_point in unvisited_points:
        candidate_point_id = int(candidate_point)
        if candidate_point_id != int(point):
            candidate_coordinates = get_location_coordinates(candidate_point_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = candidate_point_id
    return nearest_point

def get_location_coordinates(location_id, locations):
    """
    Get the coordinates of a location based on its ID.

    Args:
    - location_id: The ID of the location.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - coordinates: Tuple (x, y) representing the coordinates of the location.
    """
    for location in locations:
        if location[0] == location_id:
            return (location[1], location[2])
    return None



def calculate_distance(coordinates1, coordinates2):
    """
    Calculate the Euclidean distance between two points given their coordinates
    and round up to the nearest integer using the ceil function.

    Args:
    - coordinates1: Tuple (x1, y1) representing the coordinates of the first point.
    - coordinates2: Tuple (x2, y2) representing the coordinates of the second point.

    Returns:
    - distance: The Euclidean distance between the two points, rounded up to the nearest integer.
    """
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return ceil(distance)


def calculate_distance_between_requests(request_id1, request_id2, requests, locations):
    """
    Calculate the distance between two requests based on their corresponding location coordinates.

    Args:
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - distance: The Euclidean distance between the locations of the two requests.
    - None if either request ID is not found.
    """
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None



def get_location_details(requests, locations, request_id):
    """
    Get the location details (ID and coordinates) corresponding to the given request ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id: The request ID for which the location details are to be retrieved.

    Returns:
    - location_details: A tuple containing the location ID and its corresponding coordinates [(location_id, x_coordinate, y_coordinate)]
    - None if the request ID is not found.
    """
    for request in requests:
        if request[0] == request_id:  # Check if request ID matches
            location_id = request[1]
            for location in locations:
                if location[0] == location_id:  # Find location details by matching location ID
                    return (location_id, location[1], location[2])  # Return location details
    return None  # Return None if request ID not found

def calculate_request_distance(requests, locations, request_id1, request_id2):
    """
    Calculate the distance between two requests based on their corresponding location coordinates.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.

    Returns:
    - distance: The Euclidean distance between the two requests.
    - None if either request ID is not found.
    """
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        print(f"Location corresponding to Request ID {request_id1}: ID={location1[0]}, Coordinates=({location1[1]}, {location1[2]})")
        print(f"Location corresponding to Request ID {request_id2}: ID={location2[0]}, Coordinates=({location2[1]}, {location2[2]})")
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

# Example usage:
request_id1_input = 3
request_id2_input = 5

distance = calculate_request_distance(requests, locations, request_id1_input, request_id2_input)
if distance is not None:
    print(f"Distance between Request ID {request_id1_input} and Request ID {request_id2_input}: {distance:.2f}")
else:
    print("One or both request IDs not found.")
    
    
    
def get_requests_for_location(requests, location_id):
    """
    Get the list of request IDs corresponding to the given location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - location_id: The location ID for which the request IDs are to be retrieved.

    Returns:
    - request_ids: A list of request IDs corresponding to the given location ID.
    """
    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids

unique_location_ids = set(req[1] for req in requests)

# Iterate over unique location IDs
for location_id in unique_location_ids:
    # Get request IDs for the current location ID
    request_ids_for_location = get_requests_for_location(requests, location_id)
    # Print location ID and corresponding request IDs
    print(f"Requests belonging to Location ID {location_id}: {request_ids_for_location}")
    
    
def group_requests_by_location(requests):
    """
    Group requests by their location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]

    Returns:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    location_groups = {}
    for request in requests:
        request_id, location_id = request[0], request[1]
        if location_id not in location_groups:
            location_groups[location_id] = []
        location_groups[location_id].append(request_id)
    return location_groups

def print_location_groups(location_groups):
    """
    Print the location groups.

    Args:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    for location_id, request_ids in location_groups.items():
        print(f"Location ID {location_id}: {request_ids}")

location_groups = group_requests_by_location(requests)
print_location_groups(location_groups)



def calculate_request_weight(request_id, requests, machines):
    request = [req for req in requests if req[0] == request_id][0]
    machine_kind_id = request[4]
    num_requested_machines = request[5]
    machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
    return machine_size * num_requested_machines



def check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines):
    depot_loc_ID = depot[0]
    truck_loads = []
    current_truck_load = []
    current_truck_weight = 0
    
    for request_id, location_id, _, _, machine_kind_id, num_requested_machines in requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if current_truck_weight + request_weight > TRUCK_CAPACITY:
            if current_truck_load:
                truck_loads.append(current_truck_load)
            current_truck_load = [(request_id, request_weight)]
            current_truck_weight = request_weight
        else:
            current_truck_load.append((request_id, request_weight))
            current_truck_weight += request_weight
    

# Example usage
check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines)









Location corresponding to Request ID 3: ID=8, Coordinates=(735, 589)
Location corresponding to Request ID 5: ID=7, Coordinates=(559, 411)
Distance between Request ID 3 and Request ID 5: 251.00
Requests belonging to Location ID 8: [2, 3, 7]
Requests belonging to Location ID 3: [4]
Requests belonging to Location ID 6: [1, 6, 8, 9, 10]
Requests belonging to Location ID 7: [5]
Location ID 6: [1, 6, 8, 9, 10]
Location ID 8: [2, 3, 7]
Location ID 3: [4]
Location ID 7: [5]


In [5]:
def get_machine_type_from_request(requests):
    """
    Extract the machine type from each request and store it in a dictionary.

    Args:
    - requests: List of requests where each request is a tuple.

    Returns:
    - machine_types: Dictionary where keys are request IDs and values are machine types.
    """
    machine_types = {}
    for request in requests:
        request_id = request[0]
        machine_type = request[4]  # Assuming the machine type is the 5th entry
        machine_types[request_id] = machine_type

    return machine_types

machine_types = get_machine_type_from_request(requests)

# Print the machine types for each request
print("Machine Types for Each Request:")
for request_id, machine_type in machine_types.items():
    print(f"Request {request_id}: {machine_type}")

machine_types

Machine Types for Each Request:
Request 1: 1
Request 2: 1
Request 3: 2
Request 4: 1
Request 5: 2
Request 6: 2
Request 7: 1
Request 8: 1
Request 9: 2
Request 10: 2


{1: 1, 2: 1, 3: 2, 4: 1, 5: 2, 6: 2, 7: 1, 8: 1, 9: 2, 10: 2}

In [6]:
def get_penalty_cost_from_machine(machines):
    """
    Extract the penalty cost from each request.

    Args:
    - requests: List of requests where each request is a tuple.

    Returns:
    - penalty_cost: Dictionary where keys are request IDs and values are the penalty costs.
    """
    penalty_cost = {}
    for mach in machines:
        penalty_cost[mach[0]] = mach[2]  # Assuming the penalty cost is the 3rd entry in each request tuple
    return penalty_cost

penalty_costs_machines = get_penalty_cost_from_machine(machines)
print(penalty_costs_machines)

{1: 345, 2: 995}


In [7]:
# eerst maak method met distance tussen depot location (loc 1) en elke andere locatie

# dan maak method met distance tussen opeenvolgende requests, store dit voor alle opeenvolgende requests in array/dictionary

# maak method die kijkt van depot naar 2 requests, naar depot met afstand

# dan maak method met truck routes met dagen zoals hieronder, maar check ook of de truck geen route rijd zoals hierboven in method

In [8]:
def calculate_depot_to_request_distance(locations, depot_loc_ID, request_id):
    """
    Calculate the distance between the depot and a specific request based on their corresponding location coordinates.

    Args:
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - depot_loc_ID: The location ID of the depot.
    - request_id: The request ID of the request.

    Returns:
    - distance: The Euclidean distance between the depot and the request.
    - None if either the depot location ID or request ID is not found.
    """
    depot_coordinates = get_location_coordinates(depot_loc_ID, locations)
    request_location_id = [request[1] for request in requests if request[0] == request_id]
    if request_location_id:
        request_coordinates = get_location_coordinates(request_location_id[0], locations)
        distance = calculate_distance(depot_coordinates, request_coordinates)
        return distance
    else:
        return None

# Calculate the distances between the depot and each request
depot_loc_ID = 1  # Assuming the depot location ID is always 1
depot_to_requests = {}
for request_id in range(1, len(requests) + 1):
    distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id)
    if distance is not None:
        depot_to_requests[request_id] = distance

# Now you can access the distance for a specific request ID
print(depot_to_requests)
print(depot_to_requests[1])

{1: 226, 2: 277, 3: 277, 4: 120, 5: 51, 6: 226, 7: 277, 8: 226, 9: 226, 10: 226}
226


In [9]:
def distance_between_consecutive_requests(locations, requests, request_id1, request_id2):
    """
    Calculate the distance between two consecutive requests based on their corresponding location coordinates.

    Args:
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.

    Returns:
    - distance: The Euclidean distance between the two consecutive requests.
    - None if either request ID is not found or if the requests are not consecutive.
    """
    location_id1 = None
    location_id2 = None

    # Find the location IDs corresponding to the given request IDs
    for request in requests:
        if request[0] == request_id1:
            location_id1 = request[1]
        elif request[0] == request_id2:
            location_id2 = request[1]

    # If both location IDs are found, calculate the distance
    if location_id1 is not None and location_id2 is not None:
        coordinates1 = get_location_coordinates(location_id1, locations)
        coordinates2 = get_location_coordinates(location_id2, locations)
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

def calculate_consecutive_request_distances(locations, requests):
    """
    Calculate and store the distances between all pairs of consecutive requests.

    Args:
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]

    Returns:
    - consecutive_distances: Dictionary storing the distances between consecutive requests.
    """
    consecutive_distances = {}

    # Iterate through the requests to calculate distances between consecutive pairs
    for i in range(len(requests) - 1):
        request_id1 = requests[i][0]
        request_id2 = requests[i + 1][0]
        distance = distance_between_consecutive_requests(locations, requests, request_id1, request_id2)
        if distance is not None:
            consecutive_distances[(request_id1, request_id2)] = distance

    return consecutive_distances

# Calculate and store the distances between consecutive requests
consecutive_distances = calculate_consecutive_request_distances(locations, requests)
print(consecutive_distances)

# Print the distance between requests 2 and 3

print(consecutive_distances[(1, 2)])


{(1, 2): 246, (2, 3): 0, (3, 4): 331, (4, 5): 98, (5, 6): 176, (6, 7): 246, (7, 8): 246, (8, 9): 0, (9, 10): 0}
246


In [10]:
def combined_requests_from_and_to_depot_distance(locations, requests, depot_loc_ID, request_id1, request_id2):
    """
    Calculate the combined distances from the depot to the first request, between the two requests,
    and from the second request back to the depot.

    Args:
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - depot_loc_ID: The location ID of the depot.
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.

    Returns:
    - combined_distances: Dictionary storing the combined distances.
    """
    combined_distances = {}
    
    # Calculate distance from depot to first request
    depot_to_first_request_distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id1)
    
    # Calculate distance between the two consecutive requests
    consecutive_requests_distance = distance_between_consecutive_requests(locations, requests, request_id1, request_id2)
    
    # Calculate distance from second request back to depot
    second_request_to_depot_distance = calculate_depot_to_request_distance(locations, depot_loc_ID, request_id2)
    
    # Store the combined distances in the dictionary
    combined_distances[(request_id1, request_id2)] = {
        'depot_to_first_request_distance': depot_to_first_request_distance,
        'consecutive_requests_distance': consecutive_requests_distance,
        'second_request_to_depot_distance': second_request_to_depot_distance
    }
    
    return combined_distances

# Example usage:
combined_distances = combined_requests_from_and_to_depot_distance(locations, requests, 1, 3, 4)
print(combined_distances)

def calculate_all_combined_distances(locations, requests, depot_loc_ID):
    """
    Calculate the combined distances from the depot to the first request, between the two requests,
    and from the second request back to the depot for all pairs of consecutive requests.

    Args:
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - depot_loc_ID: The location ID of the depot.

    Returns:
    - combined_distances: Dictionary storing the combined distances for all pairs of consecutive requests.
    """
    combined_distances = {}
    
    # Iterate through all pairs of consecutive requests
    for i in range(len(requests) - 1):
        request_id1 = requests[i][0]
        request_id2 = requests[i + 1][0]
        
        # Calculate the combined distances for the current pair of consecutive requests
        combined_distance = combined_requests_from_and_to_depot_distance(locations, requests, depot_loc_ID, request_id1, request_id2)
        
        # Store the combined distances in the dictionary
        combined_distances[(request_id1, request_id2)] = combined_distance
    
    return combined_distances

# Example usage:
combined_distances = calculate_all_combined_distances(locations, requests, 1)
print(combined_distances)

# Create a dictionary to store total distances for each pair
total_distances_for_pairs = {}

# Iterate through the dictionary and calculate the total distance for each pair
for pair, distances in combined_distances.items():
    total_distance_for_pair = 0
    for sub_pair, sub_distances in distances.items():
        total_distance_for_pair += sub_distances['depot_to_first_request_distance']
        total_distance_for_pair += sub_distances['consecutive_requests_distance']
        total_distance_for_pair += sub_distances['second_request_to_depot_distance']
    total_distances_for_pairs[pair] = total_distance_for_pair

# Print or access the dictionary as needed
print(total_distances_for_pairs)



{(3, 4): {'depot_to_first_request_distance': 277, 'consecutive_requests_distance': 331, 'second_request_to_depot_distance': 120}}
{(1, 2): {(1, 2): {'depot_to_first_request_distance': 226, 'consecutive_requests_distance': 246, 'second_request_to_depot_distance': 277}}, (2, 3): {(2, 3): {'depot_to_first_request_distance': 277, 'consecutive_requests_distance': 0, 'second_request_to_depot_distance': 277}}, (3, 4): {(3, 4): {'depot_to_first_request_distance': 277, 'consecutive_requests_distance': 331, 'second_request_to_depot_distance': 120}}, (4, 5): {(4, 5): {'depot_to_first_request_distance': 120, 'consecutive_requests_distance': 98, 'second_request_to_depot_distance': 51}}, (5, 6): {(5, 6): {'depot_to_first_request_distance': 51, 'consecutive_requests_distance': 176, 'second_request_to_depot_distance': 226}}, (6, 7): {(6, 7): {'depot_to_first_request_distance': 226, 'consecutive_requests_distance': 246, 'second_request_to_depot_distance': 277}}, (7, 8): {(7, 8): {'depot_to_first_reques

In [11]:
def check_feasibility_for_all_pairs(total_distances_for_pairs, TRUCK_MAX_DISTANCE):
    """
    Check the feasibility of total distances for all pairs.

    Args:
    - total_distances_for_pairs: Dictionary where keys are pairs and values are total distances.
    - MAX_TRUCK_CAPACITY: Maximum truck capacity.

    Returns:
    - feasibility_dict: Dictionary where keys are pairs and values are boolean indicating feasibility.
    """
    feasibility_dict = {}
    for pair, total_distance in total_distances_for_pairs.items():
        feasibility_dict[pair] = total_distance <= TRUCK_MAX_DISTANCE
    return feasibility_dict

# Check feasibility for all pairs
feasibility_dict = check_feasibility_for_all_pairs(total_distances_for_pairs, TRUCK_MAX_DISTANCE)

# Print feasibility for each pair
for pair, feasible in feasibility_dict.items():
    print(f"Pair {pair}: Feasible? {feasible}")


Pair (1, 2): Feasible? False
Pair (2, 3): Feasible? True
Pair (3, 4): Feasible? False
Pair (4, 5): Feasible? True
Pair (5, 6): Feasible? True
Pair (6, 7): Feasible? False
Pair (7, 8): Feasible? False
Pair (8, 9): Feasible? True
Pair (9, 10): Feasible? True


In [12]:
def get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    current_delivery_day = None
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if not current_truck_route:
            current_truck_route.append([request_id])
            current_truck_weight = request_weight
            current_delivery_day = start_day
        elif current_truck_weight + request_weight <= TRUCK_CAPACITY and start_day == current_delivery_day:
            # Check if adding the current request exceeds the truck capacity and if delivery day matches
            combined_request_ids = current_truck_route[-1] + [request_id]
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= TRUCK_CAPACITY:
                current_truck_route[-1] = combined_request_ids
                current_truck_weight = combined_weight
            else:
                truck_routes.append((current_truck_route, current_delivery_day))
                current_truck_route = [[request_id]]
                current_truck_weight = request_weight
                current_delivery_day = start_day
        else:
            # If adding the current request exceeds the truck capacity or delivery day does not match,
            # start a new truck route with the current request
            truck_routes.append((current_truck_route, current_delivery_day))
            current_truck_route = [[request_id]]
            current_truck_weight = request_weight
            current_delivery_day = start_day
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append((current_truck_route, current_delivery_day))
    
    return truck_routes


def calculate_delivery_days(truck_routes):
    delivery_days = {}
    for truck_route, delivery_day in truck_routes:
        for request_ids in truck_route:
            for request_id in request_ids:
                delivery_days[request_id] = delivery_day
    return delivery_days



# Calculate truck routes
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)

# Calculate delivery days using truck routes
delivery_days = calculate_delivery_days(truck_routes)


print(truck_routes)



[([[1]], 2), ([[2]], 3), ([[3]], 4), ([[4]], 4), ([[5]], 4), ([[6]], 4), ([[7]], 4), ([[8]], 5), ([[9]], 5), ([[10]], 6)]


In [13]:
def get_truck_routes_with_constraints(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    current_truck_distance = 0
    current_delivery_day = None
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        # Calculate the distance from the depot to the current location
        depot_to_location_distance = calculate_request_distance(requests, locations, depot_loc_ID, location_id)
        
        # Check if the distance calculation was successful
        if depot_to_location_distance is None:
            print(f"Error: Unable to calculate distance between depot and request {request_id}")
            continue
        
        # Check if adding the current request meets all constraints
        if (current_truck_weight + request_weight <= TRUCK_CAPACITY and 
            current_truck_distance + depot_to_location_distance <= TRUCK_MAX_DISTANCE and
            start_day == current_delivery_day):
            # If constraints are satisfied, add the request to the current truck route
            if current_truck_route:
                combined_request_ids = current_truck_route[-1] + [request_id]
                if all(feasibility_dict[(req_id1, req_id2)] for req_id1 in current_truck_route[-1] for req_id2 in [request_id]):
                    current_truck_route[-1] = combined_request_ids
                    current_truck_weight += request_weight
                    current_truck_distance += depot_to_location_distance
                else:
                    truck_routes.append((current_truck_route, current_delivery_day))
                    current_truck_route = [[request_id]]  # Start a new truck route
                    current_truck_weight = request_weight
                    current_truck_distance = depot_to_location_distance
                    current_delivery_day = start_day
            else:
                current_truck_route.append([request_id])  # Add the request ID
                current_truck_weight += request_weight
                current_truck_distance += depot_to_location_distance
        else:
            # If constraints are not satisfied, start a new truck route with the current request
            if current_truck_route:
                truck_routes.append((current_truck_route, current_delivery_day))  # Append current truck route
            current_truck_route = [[request_id]]  # Start a new truck route
            current_truck_weight = request_weight
            current_truck_distance = depot_to_location_distance
            current_delivery_day = start_day
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append((current_truck_route, current_delivery_day))
    
    return truck_routes

# Calculate truck routes with constraints
truck_routes_with_constraints = get_truck_routes_with_constraints(
    locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict
)
print(truck_routes_with_constraints)



Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 6: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 8: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 8: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 3: ID=8, Coordinates=(735, 589)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 7: ID=8, Coordinates=(735, 589)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 6: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 8: ID=6, Coordinates=(490, 572)
Location corresponding to Request 

In [14]:
def constrained_truck_routes_with_depot(truck_routes_with_constraints, depot_loc_ID):
    truck_routes_with_depot = []
    for truck_route, delivery_day in truck_routes_with_constraints:
        # Add depot at the beginning and end of the truck route
        truck_route_with_depot = [depot_loc_ID] + truck_route + [depot_loc_ID]
        truck_routes_with_depot.append((truck_route_with_depot, delivery_day))
    return truck_routes_with_depot

# Calculate truck routes with constraints
truck_routes_with_constraints = get_truck_routes_with_constraints(
    locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines, feasibility_dict
)

# Add depot to truck routes
truck_routes_with_depot = constrained_truck_routes_with_depot(truck_routes_with_constraints, depot_loc_ID)

print(truck_routes_with_depot)


Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 6: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 8: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 8: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 3: ID=8, Coordinates=(735, 589)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 7: ID=8, Coordinates=(735, 589)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 6: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 1: ID=6, Coordinates=(490, 572)
Location corresponding to Request ID 8: ID=6, Coordinates=(490, 572)
Location corresponding to Request 

In [15]:
def extract_truck_routes(truck_routes_with_depot):
    extracted_routes = []
    for route, delivery_day in truck_routes_with_depot:
        request_ids = []
        for segment in route:
            if isinstance(segment, list):  # If segment is a list, extract request IDs from it
                request_ids.extend(segment)
        extracted_routes.append((request_ids, delivery_day))
    return extracted_routes


# Example usage
truck_routes_extracted = extract_truck_routes(truck_routes_with_depot)
print()
print(truck_routes_extracted)

def convert_truck_routes_to_locations(truck_routes_with_depot, requests):
    truck_routes_locations = []
    for route, delivery_day in truck_routes_with_depot:
        location_route = [1]  # Start at location ID 1
        for segment in route:
            if isinstance(segment, list):  # Check if segment is a list (request IDs)
                for request_id in segment:
                    location_id = requests[request_id - 1][1]  # Location ID of the request
                    location_route.append(location_id)
        location_route.append(1)  # End at location ID 1
        truck_routes_locations.append((location_route, delivery_day))
    return truck_routes_locations


# Convert truck routes with depot to locations
truck_routes_locations = convert_truck_routes_to_locations(truck_routes_with_depot, requests)
print()
print(truck_routes_locations)

def calculate_truck_distances(locations, truck_routes_locations):
    truck_distances_per_day = {}
    total_truck_distance = 0
    
    for idx, (route, _) in enumerate(truck_routes_locations, start=1):
        total_distance = 0
        for i in range(len(route) - 1):
            loc1 = locations[route[i] - 1]
            loc2 = locations[route[i + 1] - 1]
            distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  # Euclidean distance
            total_distance += distance
        total_distance = round(total_distance, 2)  # Round to 2 decimal places
        truck_distances_per_day[idx] = total_distance
        total_truck_distance += total_distance
    
    total_truck_distance = round(total_truck_distance, 2)  # Round total distance
    return truck_distances_per_day, total_truck_distance

# Calculate truck distances
truck_distances_per_day, total_truck_distance = calculate_truck_distances(locations, truck_routes_locations)

# Print distances for each truck per day
print("Truck Distances Per Day:")
for truck_idx, distance in truck_distances_per_day.items():
    print(f"Truck {truck_idx}: {distance:.2f}")

# Print total truck distance
print(f"\nTotal Truck Distance: {total_truck_distance:.2f}")


## weet niet of deze nog klopt na aanpassing delivery/installation days
def calculate_idle_days(requests, delivery_days):
    """
    Calculate the idle days for each machine based on delivery and installation days.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, delivery_start_day, delivery_end_day, machine_kind_id, num_requested_machines]
    - installations: List of installations where each installation is represented as a list [installation_day, machine_kind_id]

    Returns:
    - idle_days: Dictionary where keys are machine kind IDs and values are lists of idle days.
    """
    idle_days = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
        machine_kind_id = requests[request_id - 1][4]  # Machine kind ID is at index 4
        idle_days[machine_kind_id] = max(0, installation_day - delivery_day)
    return idle_days

# Calculate idle days
idle_days = calculate_idle_days(requests, delivery_days)





# Function to calculate penalties for idle days
def calculate_penalties(delivery_days, requests):
    penalties = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]
        machine_kind_id = requests[request_id - 1][4]
        idle_days = max(0, installation_day - delivery_day - 1)  # Subtract 1 for the day of delivery
        if machine_kind_id not in penalties:
            penalties[machine_kind_id] = 0
        penalties[machine_kind_id] += idle_days
    return penalties


# Function to implement constraints regarding maximum consecutive working days and total distance per day
def apply_constraints(technician_assignments):
    # Implement constraints here
    pass




[([1], 2), ([2], 3), ([3], 4), ([4], 4), ([5], 4), ([6], 4), ([7], 4), ([8], 5), ([9], 5), ([10], 6)]

[([1, 6, 1], 2), ([1, 8, 1], 3), ([1, 8, 1], 4), ([1, 3, 1], 4), ([1, 7, 1], 4), ([1, 6, 1], 4), ([1, 8, 1], 4), ([1, 6, 1], 5), ([1, 6, 1], 5), ([1, 6, 1], 6)]
Truck Distances Per Day:
Truck 1: 451.25
Truck 2: 553.81
Truck 3: 553.81
Truck 4: 239.08
Truck 5: 101.19
Truck 6: 451.25
Truck 7: 553.81
Truck 8: 451.25
Truck 9: 451.25
Truck 10: 451.25

Total Truck Distance: 4257.95


In [16]:
def check_truck_distances(truck_distances_per_day, TRUCK_MAX_DISTANCE):
    exceeded_trucks = []
    for truck_idx, distance in truck_distances_per_day.items():
        if distance > TRUCK_MAX_DISTANCE:
            exceeded_trucks.append(truck_idx)
    return exceeded_trucks

# Call this method after calculating truck distances
exceeded_trucks = check_truck_distances(truck_distances_per_day, TRUCK_MAX_DISTANCE)

if exceeded_trucks:
    print("The following trucks exceeded the maximum distance:")
    for truck_idx in exceeded_trucks:
        print(f"Truck {truck_idx}")
else:
    print("All trucks are within the maximum distance.")


All trucks are within the maximum distance.


In [17]:
def get_truck_routes_with_depot(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = [depot_loc_ID]  # Start at the depot
    current_truck_weight = 0
    current_truck_distance = 0
    current_delivery_day = None
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        # Calculate the distance from the last location in the route to the current location
        if len(current_truck_route) > 1:
            loc1 = locations[current_truck_route[-1] - 1]
            loc2 = locations[location_id - 1]
            distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  # Euclidean distance
            current_truck_distance += distance
        
        if current_truck_weight + request_weight <= TRUCK_CAPACITY and start_day == current_delivery_day and current_truck_distance <= TRUCK_MAX_DISTANCE:
            # Check if adding the current request exceeds the truck capacity and if delivery day matches
            current_truck_route.append(location_id)  # Add the location ID to the route
            current_truck_weight += request_weight
        else:
            # End the current route and start a new one
            truck_routes.append((current_truck_route, current_delivery_day))
            current_truck_route = [depot_loc_ID, location_id]  # Start a new truck route
            current_truck_weight = request_weight
            current_delivery_day = start_day
            current_truck_distance = 0
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append((current_truck_route, current_delivery_day))
    
    return truck_routes



def calculate_truck_distance(locations, route):
    total_distance = 0
    for i in range(len(route) - 1):
        loc1 = locations[route[i] - 1]
        loc2 = locations[route[i + 1] - 1]
        distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  # Euclidean distance
        total_distance += distance
    return round(total_distance, 2)

# Recalculate truck routes with depot considering maximum distance
truck_routes_with_depot = get_truck_routes_with_depot(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, requests, machines)


In [18]:
def get_location_for_request(request):
    return requests[request-1][1]

def nearest_neighbor(point, unvisited_requests, locations):

    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_request in unvisited_requests:
        request_location_id = get_location_for_request(candidate_request)
        if request_location_id != point:
            candidate_coordinates = get_location_coordinates(request_location_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = request_location_id
    return nearest_point


def get_requests_for_location(requests, location_id):
    """
    Get the list of request IDs corresponding to the given location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - location_id: The location ID for which the request IDs are to be retrieved.

    Returns:
    - request_ids: A list of request IDs corresponding to the given location ID.
    """
    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids
for location in locations:
    location_id = location[0]
    location_request  = [get_requests_for_location(requests, location_id)]
    print(location_request)
    

technician_details = {}

# Define function to print technician details
def print_technician_details(technicians):
    global technician_details
    technician_details.clear()  # Clear existing data
    for technician in technicians:
        tech_id = technician[0]
        max_distance = technician[2]
        max_installations = technician[3]
        technician_details[tech_id] = {'max_distance': max_distance, 'max_installations': max_installations}


# Call function to print technician details
print_technician_details(technicians)

# Now you can print technician_details
print(technician_details)

# Initialize max_distance and max_installations dictionaries
max_distance = {tech_id: details['max_distance'] for tech_id, details in technician_details.items()}
max_installations = {tech_id: details['max_installations'] for tech_id, details in technician_details.items()}



def initialize_home_locations(technicians):
    global home_location_technician
    home_location_technician = {tech[0]: tech[1] for tech in technicians}
    
    # Print the home locations
    print("Home Locations for Technicians:")
    for tech_id, home_location in home_location_technician.items():
        print(f"Technician {tech_id}: Location {home_location}")
    
    return home_location_technician
initialize_home_locations(technicians)
print(home_location_technician)


def nearest_neighbor_for_technician_home(technicians, unvisited_requests, locations):
    home_location_technician = initialize_home_locations(technicians)
    
    for technician_id, home_location in home_location_technician.items():
        nearest_location = nearest_neighbor(home_location, unvisited_requests, locations)
        print(f"The closest location for technician {technician_id} is location {nearest_location}.")
        
        
def transform_skill_set(technicians, technician_id):
    for technician in technicians:
        if technician[0] == technician_id:
            skill_set = technician[4:]  # Get the skill set of the technician (from index 4 onwards)
            transformed_skills = []
            
            # Iterate through the skill_set with index
            for index, value in enumerate(skill_set):
                # If the value is non-zero, append the 1-based index
                if value != 0:
                    transformed_skills.append(index + 1)
                else:
                    # If the value is zero, append zero
                    transformed_skills.append(0)
            
            return transformed_skills
    return None 

def doable_request_skillbased(technicians, requests):
    doable_requests = {}  # Dictionary to store which technician can handle which requests

    for technician in technicians:
        technician_id = technician[0]
        skill_set = transform_skill_set(technicians, technician_id)  # Get the transformed skill set of the technician
        
        for request in requests:
            request_id = request[0]
            machine_type = request[4]  # Fifth entry denotes the machine type

            # Check if the technician has this machine type in their skill set
            if machine_type in skill_set:
                if technician_id not in doable_requests:
                    doable_requests[technician_id] = []
                doable_requests[technician_id].append(request_id)
    
    return doable_requests


doable_requests = doable_request_skillbased(technicians, requests)
for technician_id, request_ids in doable_requests.items():
    print(f"Technician {technician_id} can handle requests: {request_ids}")
transformed_skills = transform_skill_set(technicians, technician_id)
if transformed_skills is not None:
    print(f"Technician {technician_id}: {transformed_skills}")
else:
    print(f"Technician {technician_id} not found.")

    
def check_location_requests_doable(technicians, requests, locations):
    # Dictionary to store the result for each location
    location_results = {}
    
    for location in locations:
        location_id = location[0]
        location_requests = [request for request in requests if request[1] == location_id]
        location_doable_requests = {}
        
        for technician in technicians:
            technician_id = technician[0]
            skill_set = transform_skill_set(technicians, technician_id)  # Get the transformed skill set of the technician
            
            doable_requests = []
            for request in location_requests:
                machine_type = request[4]  # Fifth entry denotes the machine type
                
                # Check if the technician has this machine type in their skill set
                if machine_type in skill_set:
                    doable_requests.append(request[0])
            
            if doable_requests:
                location_doable_requests[technician_id] = doable_requests
        
        location_results[location_id] = location_doable_requests
    
    return location_results

location_results = check_location_requests_doable(technicians, requests, locations)
for location_id, doable_requests in location_results.items():
    print(f"Location {location_id}:")
    for technician_id, request_ids in doable_requests.items():
        print(f"- Technician {technician_id} can handle requests: {request_ids}")

unvisited_requests = [1, 2, 3, 4,5]
current_location = 2

nearest = nearest_neighbor(current_location, unvisited_requests, locations)
print(f"The closest location for {current_location} is location {nearest}.")

# Call the new method
nearest_neighbor_for_technician_home(technicians, unvisited_requests, locations)




[[]]
[[]]
[[4]]
[[]]
[[]]
[[1, 6, 8, 9, 10]]
[[5]]
[[2, 3, 7]]
[[]]
{1: {'max_distance': 610, 'max_installations': 8}, 2: {'max_distance': 270, 'max_installations': 5}, 3: {'max_distance': 455, 'max_installations': 7}, 4: {'max_distance': 395, 'max_installations': 8}, 5: {'max_distance': 470, 'max_installations': 8}}
Home Locations for Technicians:
Technician 1: Location 9
Technician 2: Location 1
Technician 3: Location 4
Technician 4: Location 1
Technician 5: Location 1
{1: 9, 2: 1, 3: 4, 4: 1, 5: 1}
Technician 1 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Technician 2 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Technician 3 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Technician 4 can handle requests: [3, 5, 6, 9, 10]
Technician 5 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Technician 5: [1, 2]
Location 1:
Location 2:
Location 3:
- Technician 1 can handle requests: [4]
- Technician 2 can handle requests: [4]
- Technician 3 can handle requests: [4]
-

In [19]:
def extract_machine_count(requests):
    machine_count = {}
    for req in requests:
        machine_count[req[0]] = req[5]
    return machine_count


machine_count = extract_machine_count(requests)

# Print machine count for each request
print("Machine Count for Each Request:")
for request_id, count in machine_count.items():
    print(f"Request {request_id}: {count} machines")



Machine Count for Each Request:
Request 1: 3 machines
Request 2: 1 machines
Request 3: 1 machines
Request 4: 3 machines
Request 5: 2 machines
Request 6: 2 machines
Request 7: 2 machines
Request 8: 2 machines
Request 9: 2 machines
Request 10: 2 machines


In [20]:
def extract_delivery_schedule(truck_routes):
    delivery_schedule = {}
    for route in truck_routes:
        requests, delivery_day = route
        for req in requests:
            for request_id in req:
                delivery_schedule[request_id] = delivery_day
    return delivery_schedule

def print_delivery_schedule(delivery_schedule):
    print("Delivery Schedule:")
    for request_id, delivery_day in sorted(delivery_schedule.items()):
        print(f"Request {request_id}: Delivery Day {delivery_day}")


delivery_schedule = extract_delivery_schedule(truck_routes)

# Print delivery schedule for all requests
print_delivery_schedule(delivery_schedule)

# Determine the number of requests
num_requests = len(delivery_schedule)
print("Number of Requests:", num_requests)

# example to acces delivery day for 1 specific request
print("Delivery Day of Request 1:", delivery_schedule.get(1))  # Example for request 1

def extract_assignment_days(truck_routes):
    assignment_schedule = {}
    for route in truck_routes:
        requests, delivery_day = route
        for req in requests:
            for request_id in req:
                assignment_schedule[request_id] = delivery_day + 1
    return assignment_schedule

# Example usage:
assignment_schedule = extract_assignment_days(truck_routes)

# Print assignment days for all requests
print("Assignment Days:")
for request_id, assignment_day in sorted(assignment_schedule.items()):
    print(f"Request {request_id}: Assignment Day {assignment_day}")

# Example to access assignment day for a specific request
print("Assignment Day of Request 1:", assignment_schedule.get(1))  # Example for request 1





Delivery Schedule:
Request 1: Delivery Day 2
Request 2: Delivery Day 3
Request 3: Delivery Day 4
Request 4: Delivery Day 4
Request 5: Delivery Day 4
Request 6: Delivery Day 4
Request 7: Delivery Day 4
Request 8: Delivery Day 5
Request 9: Delivery Day 5
Request 10: Delivery Day 6
Number of Requests: 10
Delivery Day of Request 1: 2
Assignment Days:
Request 1: Assignment Day 3
Request 2: Assignment Day 4
Request 3: Assignment Day 5
Request 4: Assignment Day 5
Request 5: Assignment Day 5
Request 6: Assignment Day 5
Request 7: Assignment Day 5
Request 8: Assignment Day 6
Request 9: Assignment Day 6
Request 10: Assignment Day 7
Assignment Day of Request 1: 3


In [21]:
print(assignment_schedule)

{1: 3, 2: 4, 3: 5, 4: 5, 5: 5, 6: 5, 7: 5, 8: 6, 9: 6, 10: 7}


In [22]:
def doable_request_skillbased(technicians, requests):
    doable_requests = {}  # Dictionary to store which technician can handle which requests

    for technician in technicians:
        technician_id = technician[0]
        skill_set = transform_skill_set(technicians, technician_id)  # Get the transformed skill set of the technician
        
        for request in requests:
            request_id = request[0]
            machine_type = request[4]  # Fifth entry denotes the machine type

            # Check if the technician has this machine type in their skill set
            if machine_type in skill_set:
                if technician_id not in doable_requests:
                    doable_requests[technician_id] = []
                doable_requests[technician_id].append(request_id)
    
    return doable_requests


doable_requests = doable_request_skillbased(technicians, requests)

print(doable_requests[1])

for technician_id, request_ids in doable_requests.items():
    print(f"Technician {technician_id} can handle requests: {request_ids}")
    transformed_skills = transform_skill_set(technicians, technician_id)
    if transformed_skills is not None:
        print(f"Technician {technician_id}: skill set {transformed_skills}")
    else:
        print(f"Technician {technician_id} not found.")

    


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Technician 1 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Technician 1: skill set [1, 2]
Technician 2 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Technician 2: skill set [1, 2]
Technician 3 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Technician 3: skill set [1, 2]
Technician 4 can handle requests: [3, 5, 6, 9, 10]
Technician 4: skill set [0, 2]
Technician 5 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Technician 5: skill set [1, 2]


In [23]:
def home_location_technician_coordinates(home_location_technician, locations):
    """
    Get the coordinates of technicians' home locations.

    Args:
    - home_location_technician: Dictionary where keys are technician IDs and values are their home location IDs.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - home_coordinates: Dictionary where keys are technician IDs and values are tuples (x, y) representing their home location coordinates.
    """
    home_coordinates = {}
    for tech_id, location_id in home_location_technician.items():
        coordinates = get_location_coordinates(location_id, locations)
        if coordinates:
            home_coordinates[tech_id] = coordinates
    return home_coordinates

home_coordinates = home_location_technician_coordinates(home_location_technician, locations)
print("Technicians' home coordinates:")
for tech_id, coordinates in home_coordinates.items():
    print(f"Technician {tech_id}: {coordinates}")

Technicians' home coordinates:
Technician 1: (582, 711)
Technician 2: (575, 363)
Technician 3: (339, 513)
Technician 4: (575, 363)
Technician 5: (575, 363)


In [24]:

def request_location_coordinates(requests, locations):
    """
    Get the coordinates of each request location.

    Args:
    - requests: List of requests where each request is represented as [request_id, _, location_id, _, _, _].
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - request_coordinates: Dictionary where keys are request IDs and values are tuples (x, y) representing their location coordinates.
    """
    request_coordinates = {}
    for request in requests:
        request_id, location_id, *_ = request
        coordinates = get_location_coordinates(location_id, locations)
        if coordinates:
            request_coordinates[request_id] = coordinates
    return request_coordinates

request_coordinates = request_location_coordinates(requests, locations)
print("Requests' location coordinates:")
for request_id, coordinates in request_coordinates.items():
    print(f"Request {request_id}: {coordinates}")

    
def get_request_location_ids(requests):
    """
    Extract and print the location IDs of each request.

    Args:
    - requests: List of lists where each inner list represents a request.

    Returns:
    - location_ids: A list of location IDs extracted from the requests.
    """
    location_ids = [request[1] for request in requests]
    return location_ids


# Get all request location IDs
location_ids = get_request_location_ids(requests)

# Print all request location IDs
for i, location_id in enumerate(location_ids, start=1):
    print(f"Request ID {i}: Location ID {location_id}")


Requests' location coordinates:
Request 1: (490, 572)
Request 2: (735, 589)
Request 3: (735, 589)
Request 4: (462, 402)
Request 5: (559, 411)
Request 6: (490, 572)
Request 7: (735, 589)
Request 8: (490, 572)
Request 9: (490, 572)
Request 10: (490, 572)
Request ID 1: Location ID 6
Request ID 2: Location ID 8
Request ID 3: Location ID 8
Request ID 4: Location ID 3
Request ID 5: Location ID 7
Request ID 6: Location ID 6
Request ID 7: Location ID 8
Request ID 8: Location ID 6
Request ID 9: Location ID 6
Request ID 10: Location ID 6


In [25]:
def distance_between_home_and_request(home_coordinates, request_coordinates, home_id, request_id):
    """
    Calculate the distance between a home location and a request location.

    Args:
    - home_coordinates: Dictionary containing home IDs as keys and their coordinates as values.
    - request_coordinates: Dictionary containing request IDs as keys and their coordinates as values.
    - home_id: ID of the home.
    - request_id: ID of the request.

    Returns:
    - distance: The Euclidean distance between the home and request locations.
    """
    home_location = home_coordinates.get(home_id)
    request_location = request_coordinates.get(request_id)
    if home_location is None:
        raise ValueError(f"Home ID {home_id} not found in home_coordinates.")
    if request_location is None:
        raise ValueError(f"Request ID {request_id} not found in request_coordinates.")
    
    return calculate_distance(home_location, request_location)

def print_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates):
    """
    Print distances between each technician's home and each request.

    Args:
    - home_coordinates: Dictionary containing home IDs as keys and their coordinates as values.
    - request_coordinates: Dictionary containing request IDs as keys and their coordinates as values.
    """
    for home_id, home_location in home_coordinates.items():
        for request_id, request_location in request_coordinates.items():
            distance = distance_between_home_and_request(home_coordinates, request_coordinates, home_id, request_id)
            print(f"Distance between home {home_id} and request {request_id}: {distance}")

print_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates)

Distance between home 1 and request 1: 167
Distance between home 1 and request 2: 196
Distance between home 1 and request 3: 196
Distance between home 1 and request 4: 332
Distance between home 1 and request 5: 301
Distance between home 1 and request 6: 167
Distance between home 1 and request 7: 196
Distance between home 1 and request 8: 167
Distance between home 1 and request 9: 167
Distance between home 1 and request 10: 167
Distance between home 2 and request 1: 226
Distance between home 2 and request 2: 277
Distance between home 2 and request 3: 277
Distance between home 2 and request 4: 120
Distance between home 2 and request 5: 51
Distance between home 2 and request 6: 226
Distance between home 2 and request 7: 277
Distance between home 2 and request 8: 226
Distance between home 2 and request 9: 226
Distance between home 2 and request 10: 226
Distance between home 3 and request 1: 163
Distance between home 3 and request 2: 404
Distance between home 3 and request 3: 404
Distance b

In [26]:
def calculate_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates):
    """
    Calculate and store distances between each technician's home and each request.

    Args:
    - home_coordinates: Dictionary containing home IDs as keys and their coordinates as values.
    - request_coordinates: Dictionary containing request IDs as keys and their coordinates as values.

    Returns:
    - distances: Dictionary with keys as (home_id, request_id) tuples and values as distances.
    """
    distances = {}
    for home_id, home_location in home_coordinates.items():
        for request_id, request_location in request_coordinates.items():
            distance = distance_between_home_and_request(home_coordinates, request_coordinates, home_id, request_id)
            distances[(home_id, request_id)] = distance
            print(f"Distance between home {home_id} and request {request_id}: {distance}")
    return distances


distances = calculate_distances_for_all_technicians_and_requests(home_coordinates, request_coordinates)



Distance between home 1 and request 1: 167
Distance between home 1 and request 2: 196
Distance between home 1 and request 3: 196
Distance between home 1 and request 4: 332
Distance between home 1 and request 5: 301
Distance between home 1 and request 6: 167
Distance between home 1 and request 7: 196
Distance between home 1 and request 8: 167
Distance between home 1 and request 9: 167
Distance between home 1 and request 10: 167
Distance between home 2 and request 1: 226
Distance between home 2 and request 2: 277
Distance between home 2 and request 3: 277
Distance between home 2 and request 4: 120
Distance between home 2 and request 5: 51
Distance between home 2 and request 6: 226
Distance between home 2 and request 7: 277
Distance between home 2 and request 8: 226
Distance between home 2 and request 9: 226
Distance between home 2 and request 10: 226
Distance between home 3 and request 1: 163
Distance between home 3 and request 2: 404
Distance between home 3 and request 3: 404
Distance b

In [27]:
print(home_location_technician)
print(home_coordinates)
print(home_coordinates.get(1))
print(request_coordinates)
distances[(1,9)]


{1: 9, 2: 1, 3: 4, 4: 1, 5: 1}
{1: (582, 711), 2: (575, 363), 3: (339, 513), 4: (575, 363), 5: (575, 363)}
(582, 711)
{1: (490, 572), 2: (735, 589), 3: (735, 589), 4: (462, 402), 5: (559, 411), 6: (490, 572), 7: (735, 589), 8: (490, 572), 9: (490, 572), 10: (490, 572)}


167

In [28]:
def check_max_distance_exceeded(distances, max_distance):
    """
    Check if the round trip distance for each technician exceeds their maximum distance.

    Args:
    - distances: Dictionary containing distances between each technician's home and each request.
    - max_distance: Dictionary containing maximum distances for each technician.

    Returns:
    - results: Dictionary containing tuples of (home_id, request_id) as keys and boolean values indicating
               whether the round trip distance exceeds the maximum distance for the corresponding technician.
    """
    results = {}
    for (home_id, request_id), distance in distances.items():
        round_trip_distance = 2 * distance
        max_dist = max_distance.get(home_id, float('inf'))
        exceeds_max_distance = round_trip_distance > max_dist
        results[(home_id, request_id)] = exceeds_max_distance
    return results

# Assuming distances is already calculated from the previous code
exceeded_distances = check_max_distance_exceeded(distances, max_distance)

# Print the results
for (home_id, request_id), exceeds in exceeded_distances.items():
    status = "exceeds" if exceeds else "does not exceed"
    print(f"Round trip distance for home {home_id} to request {request_id} {'exceeds' if exceeds else 'does not exceed'} the max distance.")

    
## ONDERSTAANDE GEBRUIKT OM TE CHECKEN OF HET ECHT EXCEEDE OF NIET MET PRINTEN VAN DISTANCE AND MAX DISTANCE, MAG WEG
## def check_max_distance_exceeded(distances, max_distance):
##    """
##    Check if the round trip distance for each technician exceeds their maximum distance.

##      Args:
##      - distances: Dictionary containing distances between each technician's home and each request.
##      - max_distance: Dictionary containing maximum distances for each technician.

##      Returns:
##      - results: Dictionary containing tuples of (home_id, request_id) as keys and boolean values indicating
##                 whether the round trip distance exceeds the maximum distance for the corresponding technician.
##      """
##      results = {}
##      for (home_id, request_id), distance in distances.items():
##          round_trip_distance = 2 * distance
##          max_dist = max_distance.get(home_id, float('inf'))
##          exceeds_max_distance = round_trip_distance > max_dist
##          results[(home_id, request_id)] = exceeds_max_distance
##      return results

##  # Assuming distances is already calculated from the previous code
##  exceeded_distances = check_max_distance_exceeded(distances, max_distance)

##  # Print the results
##  for (home_id, request_id), exceeds in exceeded_distances.items():
##      status = "exceeds" if exceeds else "does not exceed"
##      distance = distances[(home_id, request_id)]
##      max_dist = max_distance.get(home_id, float('inf'))
##      round_trip_distance = distance*2
##      exceeds_max_distance = round_trip_distance > max_dist
##      print(f"Round trip distance for home {home_id} to request {request_id} (distance is {round_trip_distance}) {'exceeds' if exceeds_max_distance else 'does not exceed'} the max distance (max distance of {max_dist}).")


Round trip distance for home 1 to request 1 does not exceed the max distance.
Round trip distance for home 1 to request 2 does not exceed the max distance.
Round trip distance for home 1 to request 3 does not exceed the max distance.
Round trip distance for home 1 to request 4 exceeds the max distance.
Round trip distance for home 1 to request 5 does not exceed the max distance.
Round trip distance for home 1 to request 6 does not exceed the max distance.
Round trip distance for home 1 to request 7 does not exceed the max distance.
Round trip distance for home 1 to request 8 does not exceed the max distance.
Round trip distance for home 1 to request 9 does not exceed the max distance.
Round trip distance for home 1 to request 10 does not exceed the max distance.
Round trip distance for home 2 to request 1 exceeds the max distance.
Round trip distance for home 2 to request 2 exceeds the max distance.
Round trip distance for home 2 to request 3 exceeds the max distance.
Round trip distan

In [29]:
exceeded_distances

{(1, 1): False,
 (1, 2): False,
 (1, 3): False,
 (1, 4): True,
 (1, 5): False,
 (1, 6): False,
 (1, 7): False,
 (1, 8): False,
 (1, 9): False,
 (1, 10): False,
 (2, 1): True,
 (2, 2): True,
 (2, 3): True,
 (2, 4): False,
 (2, 5): False,
 (2, 6): True,
 (2, 7): True,
 (2, 8): True,
 (2, 9): True,
 (2, 10): True,
 (3, 1): False,
 (3, 2): True,
 (3, 3): True,
 (3, 4): False,
 (3, 5): True,
 (3, 6): False,
 (3, 7): True,
 (3, 8): False,
 (3, 9): False,
 (3, 10): False,
 (4, 1): True,
 (4, 2): True,
 (4, 3): True,
 (4, 4): False,
 (4, 5): False,
 (4, 6): True,
 (4, 7): True,
 (4, 8): True,
 (4, 9): True,
 (4, 10): True,
 (5, 1): False,
 (5, 2): True,
 (5, 3): True,
 (5, 4): False,
 (5, 5): False,
 (5, 6): False,
 (5, 7): True,
 (5, 8): False,
 (5, 9): False,
 (5, 10): False}

In [30]:
def doable_requests_distance_based(exceeded_distances):
    """
    Check which requests each technician can handle based on the distances and maximum distances.

    Args:
    - exceeded_distances: Dictionary containing tuples of (home_id, request_id) as keys and boolean values indicating
                          whether the round trip distance exceeds the maximum distance for the corresponding technician.

    Returns:
    - doable_requests: Dictionary where keys are technician ids and values are lists of requests that the technician can handle.
    """
    doable_requests = {}
    for (home_id, request_id), exceeds in exceeded_distances.items():
        if not exceeds:
            if home_id not in doable_requests:
                doable_requests[home_id] = []
            doable_requests[home_id].append(request_id)
    return doable_requests

# Get doable requests based on distance
doable_requests_distance_based = doable_requests_distance_based(exceeded_distances)

# Print the result
for technician, requests in doable_requests_distance_based.items():
    print(f"Technician {technician} can handle requests: {requests}")


Technician 1 can handle requests: [1, 2, 3, 5, 6, 7, 8, 9, 10]
Technician 2 can handle requests: [4, 5]
Technician 3 can handle requests: [1, 4, 6, 8, 9, 10]
Technician 4 can handle requests: [4, 5]
Technician 5 can handle requests: [1, 4, 5, 6, 8, 9, 10]


In [31]:
# dit returned een list met requests die de technicain kan doen based op zn skills en op zn distance vanaf home


def doable_requests_double_check(doable_requests_distance_based, doable_requests):
    double_check_requests = {}

    # Iterate through technician IDs
    for tech_id in doable_requests_distance_based.keys():
        # Find common requests between both dictionaries
        common_requests = set(doable_requests_distance_based[tech_id]) & set(doable_requests[tech_id])
        double_check_requests[tech_id] = list(common_requests)

    return double_check_requests



double_check_requests = doable_requests_double_check(doable_requests_distance_based, doable_requests)

print(double_check_requests)

{1: [1, 2, 3, 5, 6, 7, 8, 9, 10], 2: [4, 5], 3: [1, 4, 6, 8, 9, 10], 4: [5], 5: [1, 4, 5, 6, 8, 9, 10]}


In [32]:
def find_requests_with_same_day_and_loc(assignment_schedule, request_coordinates, location_ids):
    """
    Find requests with the same location ID and assignment day.

    Args:
    - assignment_schedule: Dictionary where keys are request IDs and values are assignment days.
    - request_coordinates: Dictionary where keys are request IDs and values are tuples representing location coordinates.
    - location_ids: List of location IDs extracted from the requests.

    Returns:
    - requests_same_day_loc: Dictionary where keys are tuples (location ID, assignment day) and values are lists of request IDs.
    """
    requests_same_day_loc = {}
    for request_id, assignment_day in assignment_schedule.items():
        location_id = location_ids[request_id - 1]  # Adjusting index since request_id starts from 1
        if location_id is not None:
            key = (location_id, assignment_day)
            requests_same_day_loc.setdefault(key, []).append(request_id)
    return {key: requests for key, requests in requests_same_day_loc.items() if len(requests) > 1}

# Example usage:
requests_same_day_loc = find_requests_with_same_day_and_loc(assignment_schedule, request_coordinates, location_ids)

# Print requests with the same location and assignment day
print("Requests with the Same Location ID and Assignment Day:")
for (location_id, assignment_day), requests in requests_same_day_loc.items():
    print(f"Location ID {location_id}, Assignment Day {assignment_day}: Requests {requests}")


Requests with the Same Location ID and Assignment Day:
Location ID 8, Assignment Day 5: Requests [3, 7]
Location ID 6, Assignment Day 6: Requests [8, 9]


In [33]:
requests_same_day_loc

{(8, 5): [3, 7], (6, 6): [8, 9]}

In [34]:
# method om te kijken of pairs in find_requests_with_same_day_and_loc are also pairs in doable_requests_double_check
def check_tech_req_same_day(double_check_requests, request_coordinates, assignment_schedule, location_ids):
    """
    Check if technicians can do requests with the same location and assignment day.

    Args:
    - double_check_requests: Dictionary where keys are technician IDs and values are lists of doable request IDs.
    - request_coordinates: Dictionary where keys are request IDs and values are tuples representing location coordinates.
    - assignment_schedule: Dictionary where keys are request IDs and values are assignment days.
    - location_ids: List of location IDs extracted from the requests.

    Returns:
    - tech_req_same_day: Dictionary where keys are tuples (technician ID, assignment day) and values are lists of request IDs.
    """
    tech_req_same_day = {}

    # Iterate through technician IDs
    for tech_id, requests in double_check_requests.items():
        # Initialize a dictionary to store requests with the same location and assignment day for the technician
        req_same_day = {}

        # Iterate over the technician's double-check requests
        for req_id in requests:
            location_id = location_ids[req_id - 1]  # Adjusting index since req_id starts from 1
            day = assignment_schedule[req_id]

            # Create a key for the location and assignment day
            key = (location_id, day)

            # Add the request to the dictionary under the corresponding key
            req_same_day.setdefault(key, []).append(req_id)

        # Filter out requests with the same location and assignment day
        req_same_day = {key: req_ids for key, req_ids in req_same_day.items() if len(req_ids) > 1}

        # Add the filtered dictionary to the tech_req_same_day dictionary
        if req_same_day:
            tech_req_same_day[tech_id] = req_same_day

    return tech_req_same_day

# Example usage:
tech_req_same_day = check_tech_req_same_day(double_check_requests, request_coordinates, assignment_schedule, location_ids)

# Print technicians who can do requests with the same location and assignment day
print("Technicians and Requests with the Same Location ID and Assignment Day:")
for tech_id, req_same_day in tech_req_same_day.items():
    for (location_id, assignment_day), req_ids in req_same_day.items():
        print(f"Technician ID {tech_id}, Location ID {location_id}, Assignment Day {assignment_day}: Requests {req_ids}")


Technicians and Requests with the Same Location ID and Assignment Day:
Technician ID 1, Location ID 8, Assignment Day 5: Requests [3, 7]
Technician ID 1, Location ID 6, Assignment Day 6: Requests [8, 9]
Technician ID 3, Location ID 6, Assignment Day 6: Requests [8, 9]
Technician ID 5, Location ID 6, Assignment Day 6: Requests [8, 9]


In [35]:
def possible_tech_combis_on_day(tech_req_same_day):
    """
    Create pairs of requests for technicians who can handle requests with the same location ID and assignment day.

    Args:
    - tech_req_same_day: Dictionary where keys are technician IDs and values are dictionaries of requests with the same location and assignment day.

    Returns:
    - tech_combis: List of tuples where each tuple contains a pair of requests for a technician.
    """
    tech_combis = []

    # Iterate through technicians
    for tech_id, req_same_day in tech_req_same_day.items():
        # Iterate through requests with the same location and assignment day
        for req_ids in req_same_day.values():
            # Create pairs of consecutive requests
            for i in range(len(req_ids) - 1):
                tech_combis.append((tech_id, req_ids[i], req_ids[i + 1]))

    return tech_combis

# Example usage:
tech_combis = possible_tech_combis_on_day(tech_req_same_day)

# Print possible technician pairs with consecutive requests
print("Possible Technician Pairs with Consecutive Requests:")
for tech_id, req1, req2 in tech_combis:
    print(f"Technician ID {tech_id}: Requests {req1} and {req2}")
    
tech_combis


Possible Technician Pairs with Consecutive Requests:
Technician ID 1: Requests 3 and 7
Technician ID 1: Requests 8 and 9
Technician ID 3: Requests 8 and 9
Technician ID 5: Requests 8 and 9


[(1, 3, 7), (1, 8, 9), (3, 8, 9), (5, 8, 9)]

In [36]:
def weighted_tech_combis(tech_combis, machine_count):
    """
    Calculate the weight of each tech combi based on the machine count of the requests it contains.

    Args:
    - tech_combis: List of tuples where each tuple contains a tech ID and two request IDs.
    - machine_count: Dictionary where keys are request IDs and values are the machine count.

    Returns:
    - weighted_combis: Dictionary where keys are tech combis (tuples) and values are their weights.
    """
    weighted_combis = {}
    
    # Iterate through each tech combi
    for tech_id, req1, req2 in tech_combis:
        # Calculate the weight of the tech combi by summing the machine counts of the requests it contains
        weight = machine_count.get(req1, 0) + machine_count.get(req2, 0)
        weighted_combis[(tech_id, req1, req2)] = weight
    
    return weighted_combis

# Example usage:
weighted_combis = weighted_tech_combis(tech_combis, machine_count)

# Print weighted tech combis
print("Weighted Tech Combis:")
for combi, weight in weighted_combis.items():
    print(f"Tech Combi {combi}: Weight {weight}")

weighted_combis

Weighted Tech Combis:
Tech Combi (1, 3, 7): Weight 3
Tech Combi (1, 8, 9): Weight 4
Tech Combi (3, 8, 9): Weight 4
Tech Combi (5, 8, 9): Weight 4


{(1, 3, 7): 3, (1, 8, 9): 4, (3, 8, 9): 4, (5, 8, 9): 4}

In [37]:
def feasible_tech_combi(weighted_combis, max_installations):
    """
    Check if each tech combi is feasible for a technician based on their maximum installations per day.

    Args:
    - weighted_combis: Dictionary with tech combis as keys and their weights as values.
    - max_installations: Dictionary where keys are technician IDs and values are their maximum installations.

    Returns:
    - feasible_combis: Dictionary with tech combis as keys and boolean values indicating feasibility.
    """
    feasible_combis = {}
    for tech_combi, combi_weight in weighted_combis.items():
        tech_id = tech_combi[0]
        max_installation = max_installations.get(tech_id, 0)
        feasible_combis[tech_combi] = combi_weight <= max_installation
    return feasible_combis


feasible_combis = feasible_tech_combi(weighted_combis, max_installations)

# Print the feasibility of each tech combi
for tech_combi, is_feasible in feasible_combis.items():
    print(f"Is tech combi {tech_combi} feasible? {is_feasible}")


Is tech combi (1, 3, 7) feasible? True
Is tech combi (1, 8, 9) feasible? True
Is tech combi (3, 8, 9) feasible? True
Is tech combi (5, 8, 9) feasible? True


In [38]:
def assign_requests(assignment_schedule, double_check_requests, days, feasible_combis):
    # Create a dictionary to store the assigned requests for each technician
    assigned_requests = {}
    
    # Create a dictionary to track the consecutive days each technician has worked
    consecutive_days = {tech_id: 0 for tech_id in double_check_requests.keys()}

    # Create a dictionary to store the requests assigned on a later day
    later_deliveries = {}

    # Create a set to store requests that could not be assigned
    unassigned_requests = set()

    # Convert feasible_combis to a dictionary for easy lookup
    feasible_combis_dict = {}
    for combi, is_feasible in feasible_combis.items():
        if is_feasible:
            tech_id, req1, req2 = combi
            feasible_combis_dict.setdefault(tech_id, []).append((req1, req2))

    # Create a set to store requests that have already been assigned
    assigned_request_ids = set()

    # Iterate through each request in the assignment schedule
    for request_id, assignment_day in sorted(assignment_schedule.items(), key=lambda x: x[1]):
        # Skip this request if it's already been assigned
        if request_id in assigned_request_ids:
            continue
        
        assigned = False
        for day_delay in range(days + 1):
            day_to_assign = assignment_day + day_delay
            # Check if the day to assign exceeds the maximum allowed day
            if day_to_assign > days:
                # Mark the request as unassigned and break the loop
                unassigned_requests.add(request_id)
                break

            # Find technicians who can work on this request
            available_technicians = [tech_id for tech_id, requests in double_check_requests.items() if request_id in requests]

            if not available_technicians:
                continue

            # Find the earliest available technician for this request with the smallest number of doable requests
            min_consecutive_days = min(consecutive_days[tech_id] for tech_id in available_technicians)
            suitable_technicians = [tech_id for tech_id in available_technicians if consecutive_days[tech_id] <= min_consecutive_days]
            
            if not suitable_technicians:
                continue

            # Select the technician with the smallest number of doable requests
            min_assigned = min(suitable_technicians, key=lambda x: len(double_check_requests[x]))

            # Check if the technician is already assigned a request on this day
            if any(day == day_to_assign for _, day in assigned_requests.get(min_assigned, [])):
                # Find another available technician for this request
                suitable_technicians.remove(min_assigned)
                if not suitable_technicians:
                    continue
                min_assigned = min(suitable_technicians, key=lambda x: len(double_check_requests[x]))

            # Assign the request to the technician
            assigned_requests.setdefault(min_assigned, []).append((request_id, day_to_assign))
            assigned_request_ids.add(request_id)

            # If the request was assigned on a later day, record the delay
            if day_to_assign > assignment_day:
                later_deliveries[request_id] = day_to_assign - assignment_day

                # Check if this request is part of a feasible combi
                if min_assigned in feasible_combis_dict:
                    for req1, req2 in feasible_combis_dict[min_assigned]:
                        if request_id == req1 and req2 not in assigned_request_ids:
                            # Add the other request from the combi to later_deliveries
                            later_deliveries[req2] = day_to_assign - assignment_day
                            break
                        elif request_id == req2 and req1 not in assigned_request_ids:
                            # Add the other request from the combi to later_deliveries
                            later_deliveries[req1] = day_to_assign - assignment_day
                            break

            # Check for a feasible combi with this request
            combi_assigned = False
            if min_assigned in feasible_combis_dict:
                for req1, req2 in feasible_combis_dict[min_assigned]:
                    if request_id == req1 and req2 not in assigned_request_ids:
                        # Assign the other request from the combi
                        assigned_requests[min_assigned].append((req2, day_to_assign))
                        assigned_request_ids.add(req2)
                        combi_assigned = True
                        break
                    elif request_id == req2 and req1 not in assigned_request_ids:
                        # Assign the other request from the combi
                        assigned_requests[min_assigned].append((req1, day_to_assign))
                        assigned_request_ids.add(req1)
                        combi_assigned = True
                        break

                if combi_assigned:
                    # Remove the assigned combi from feasible_combis_dict
                    feasible_combis_dict[min_assigned] = [(req1, req2) for req1, req2 in feasible_combis_dict[min_assigned]
                                                           if req1 not in assigned_request_ids and req2 not in assigned_request_ids]

            # Update consecutive days for the assigned technician
            consecutive_days[min_assigned] += 1

            # Reset consecutive days for technicians who are not assigned on this day
            for tech_id in double_check_requests.keys():
                if tech_id != min_assigned:
                    consecutive_days[tech_id] = 0

            assigned = True
            break

        if not assigned:
            if assignment_day <= days:
                unassigned_requests.add(request_id)

    return assigned_requests, later_deliveries, unassigned_requests




feasible_combis = feasible_tech_combi(weighted_combis, max_installations)

assigned_requests, later_deliveries, unassigned_requests = assign_requests(assignment_schedule, double_check_requests, DAYS, feasible_combis)

# Print the assigned requests
for technician, requests in assigned_requests.items():
    print(f"Technician {technician} assigned requests:")
    for request, day in requests:
        print(f"Request {request} assigned on day {day}")

# Print the later deliveries
print("\nLater deliveries:")
for request_id, delay in later_deliveries.items():
    print(f"Request {request_id} assigned {delay} day(s) later")

# Print the unassigned requests
print("\nUnassigned requests:")
for request_id in unassigned_requests:
    print(f"Request {request_id} could not be assigned within {DAYS} days")


Technician 3 assigned requests:
Request 1 assigned on day 3
Request 6 assigned on day 5
Request 10 assigned on day 7
Technician 1 assigned requests:
Request 2 assigned on day 4
Request 3 assigned on day 5
Request 7 assigned on day 5
Technician 2 assigned requests:
Request 4 assigned on day 5
Technician 4 assigned requests:
Request 5 assigned on day 5
Technician 5 assigned requests:
Request 8 assigned on day 6
Request 9 assigned on day 6

Later deliveries:

Unassigned requests:


In [39]:
def idle_request_days(later_deliveries):
    """
    Calculate the number of days each request is assigned later than its scheduled day.

    Args:
    - later_deliveries: Dictionary where keys are request IDs and values are the number of days delayed.

    Returns:
    - idle_days: Dictionary where keys are request IDs and values are the number of days each request is late.
    """
    idle_days = {}
    for request_id, delay in later_deliveries.items():
        idle_days[request_id] = delay

    return idle_days


# Example usage with the previous results
idle_days = idle_request_days(later_deliveries)

# Print idle request days
print("\nIdle request days:")
for request_id, days_late in idle_days.items():
    print(f"Request {request_id} is {days_late} day(s) too late")

idle_days



Idle request days:


{}

In [40]:
# read again
"""
Enter filepath of instance and desired name for the solution file
"""
directory = "/Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project"
filename = filename
output_file_name = output_file_name


"""
 Read the instance and assign variables to the different values
"""

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}

print("Machines:")
print(machines)

print("\nLocations:")
print(locations)

print("\nRequests:")
print(requests)

print("\nTechnicians:")
print(technicians)


print(DAYS)
print(TRUCK_CAPACITY)
print(TRUCK_MAX_DISTANCE)

print(TRUCK_DISTANCE_COST)
print(TRUCK_DAY_COST)
print(TRUCK_COST)
print(TECHNICIAN_DISTANCE_COST)
print(TECHNICIAN_DAY_COST)
print(TECHNICIAN_COST)

Machines:
[[  1   3 345]
 [  2   7 995]]

Locations:
[[  1 575 363]
 [  2 649 337]
 [  3 462 402]
 [  4 339 513]
 [  5 365 182]
 [  6 490 572]
 [  7 559 411]
 [  8 735 589]
 [  9 582 711]]

Requests:
[[ 1  6  2  3  1  3]
 [ 2  8  3  4  1  1]
 [ 3  8  4  7  2  1]
 [ 4  3  4  5  1  3]
 [ 5  7  4  7  2  2]
 [ 6  6  4  7  2  2]
 [ 7  8  4  6  1  2]
 [ 8  6  5  7  1  2]
 [ 9  6  5  7  2  2]
 [10  6  6  7  2  2]]

Technicians:
[[  1   9 610   8   1   1]
 [  2   1 270   5   1   1]
 [  3   4 455   7   1   1]
 [  4   1 395   8   0   1]
 [  5   1 470   8   1   1]]
10
15
555
0
0
0
10
0
0


In [41]:
def get_idle_days_per_machine(idle_days, requests):
    """
    Calculate the total idle days for each machine type.

    Args:
    - idle_days: Dictionary where keys are request IDs and values are the number of days each request is late.
    - requests: List of requests where each request is a tuple.

    Returns:
    - machine_idle_days: Dictionary where keys are machine types and values are the total number of idle days.
    """
    # Get machine types from requests
    machine_types = get_machine_type_from_request(requests)
    
    # Initialize dictionary to store idle days for each machine type
    machine_idle_days = {}

    # Iterate through the idle days dictionary
    for request_id, days_late in idle_days.items():
        # Get the machine type for this request
        machine_type = machine_types.get(request_id)
        
        if machine_type:
            # Add the idle days to the corresponding machine type
            if machine_type in machine_idle_days:
                machine_idle_days[machine_type] += days_late
            else:
                machine_idle_days[machine_type] = days_late

    return machine_idle_days


machine_idle_days = get_idle_days_per_machine(idle_days, requests)

# Print the total idle days for each machine type
print("Total Idle Days for Each Machine Type:")
for machine_type, total_idle_days in machine_idle_days.items():
    print(f"{machine_type}: {total_idle_days} day(s)")


Total Idle Days for Each Machine Type:


In [42]:
def calculate_idle_costs(machine_idle_days, penalty_costs_machines):
    """
    Calculate the costs of idle machine days.

    Args:
    - machine_idle_days: Dictionary where keys are machine types and values are the total number of idle days.
    - penalty_costs_machines: Dictionary where keys are machine types and values are the penalty costs per day.

    Returns:
    - idle_costs: Dictionary where keys are machine types and values are the total costs of idle days.
    """
    idle_costs = {}
    for machine_type, idle_days in machine_idle_days.items():
        penalty_cost = penalty_costs_machines.get(machine_type, 0)
        idle_costs[machine_type] = idle_days * penalty_cost
    return idle_costs

# Example usage:
idle_costs = calculate_idle_costs(machine_idle_days, penalty_costs_machines)

# Print the total costs of idle days for each machine type
print("Total Costs of Idle Days for Each Machine Type:")
total_idle_costs = 0
for machine_type, total_costs in idle_costs.items():
    print(f"{machine_type}: {total_costs} cost units")
    total_idle_costs += total_costs

print(f"Total Idle Costs: {total_idle_costs} cost ")


Total Costs of Idle Days for Each Machine Type:
Total Idle Costs: 0 cost 


In [43]:
def format_assigned_requests(assigned_requests, current_day=None):
    formatted_output = ""
    
    # Group requests by day
    requests_by_day = {}
    for technician, requests in assigned_requests.items():
        for request, day in requests:
            requests_by_day.setdefault(day, []).append((technician, request))
    
    # If current_day is provided, filter to only that day
    if current_day is not None:
        sorted_days = [current_day]
    else:
        sorted_days = sorted(requests_by_day.keys())
    
    # Iterate through each day
    for day in sorted_days:
        formatted_output += f"DAY {day}\n"
        requests_on_day = requests_by_day.get(day, [])
        
        # Count number of technicians
        num_technicians = len(set(technician for technician, _ in requests_on_day))
        formatted_output += f"NUMBER_OF_TECHNICIANS = {num_technicians}\n"
        
        # Group requests by technician
        requests_by_technician = {}
        for technician, request in requests_on_day:
            requests_by_technician.setdefault(technician, []).append(request)
        
        # Format requests for each technician
        for technician, requests in requests_by_technician.items():
            formatted_output += f"{technician} {' '.join(str(request) for request in requests)}\n"
        
        formatted_output += "\n"  # Add a newline between days
    
    return formatted_output

# Example usage
formatted_output = format_assigned_requests(assigned_requests)
print(formatted_output)

DAY 3
NUMBER_OF_TECHNICIANS = 1
3 1

DAY 4
NUMBER_OF_TECHNICIANS = 1
1 2

DAY 5
NUMBER_OF_TECHNICIANS = 4
3 6
1 3 7
2 4
4 5

DAY 6
NUMBER_OF_TECHNICIANS = 1
5 8 9

DAY 7
NUMBER_OF_TECHNICIANS = 1
3 10




In [44]:
def store_technician_routes(assigned_requests):
    technician_routes = {}
    
    # Group requests by technician
    for technician, requests in assigned_requests.items():
        # Sort requests by day
        sorted_requests = sorted(requests, key=lambda x: x[1])
        
        # Initialize the route for the technician
        technician_routes[technician] = [[] for _ in range(max(day for _, day in sorted_requests) + 1)]
        
        # Fill in the route
        for request, day in sorted_requests:
            technician_routes[technician][day].append(request)
    
    return technician_routes

# Example usage
technician_routes = store_technician_routes(assigned_requests)

# Print the technician routes
for technician, route in technician_routes.items():
    print(f"Technician {technician} route:")
    for day, requests in enumerate(route):
        if requests:  # Check if the array is non-empty
            print(f"DAY {day}: {requests}")
    print()


Technician 3 route:
DAY 3: [1]
DAY 5: [6]
DAY 7: [10]

Technician 1 route:
DAY 4: [2]
DAY 5: [3, 7]

Technician 2 route:
DAY 5: [4]

Technician 4 route:
DAY 5: [5]

Technician 5 route:
DAY 6: [8, 9]



In [45]:
def store_all_technician_routes(technician_routes):
    all_tech_routes = []
    for technician, route in technician_routes.items():
        for requests in route:
            all_tech_routes.extend([(technician, request) for request in requests])
    return all_tech_routes

# Example usage
all_tech_routes = store_all_technician_routes(technician_routes)

# Print all technician routes
print("All technician routes:")
print(all_tech_routes)


All technician routes:
[(3, 1), (3, 6), (3, 10), (1, 2), (1, 3), (1, 7), (2, 4), (4, 5), (5, 8), (5, 9)]


In [46]:
def calculate_distances_for_all_technician_routes(all_tech_routes, distances):
    tech_route_distances = {}
    for tech, req in all_tech_routes:
        distance = distances[(tech, req)]
        if tech not in tech_route_distances:
            tech_route_distances[tech] = 0
        tech_route_distances[tech] += distance
    return tech_route_distances

# Example usage
tech_route_distances = calculate_distances_for_all_technician_routes(all_tech_routes, distances)

# Print technician route distances
print("Technician route distances:")
for technician, distance in tech_route_distances.items():
    print(f"Technician {technician} total distance: {distance}")
    
def calculate_total_distance_for_all_technicians(tech_route_distances):
    total_distance = sum(tech_route_distances.values())
    return total_distance

# Example usage
total_distance_all_techs = calculate_total_distance_for_all_technicians(tech_route_distances)

# Print the total distance for all technicians
print(f"Total distance for all technicians: {total_distance_all_techs}")



Technician route distances:
Technician 3 total distance: 489
Technician 1 total distance: 588
Technician 2 total distance: 120
Technician 4 total distance: 51
Technician 5 total distance: 452
Total distance for all technicians: 1700


In [47]:
def print_schedule_for_each_day(truck_routes, delivery_days):
    all_days = sorted(set(delivery_days.values()))  # Get all unique delivery days
    
    for current_day in all_days:
        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")
        
        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        
        if num_trucks_for_day > 0:
            print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
            for truck_index, route in enumerate(truck_routes_for_day, start=1):
                print(truck_index, end=' ')
                for request_id in route:
                    print(request_id, end=' ')  # Print each request ID in the route
                print()  # Newline after each truck route
        else:
            print("NUMBER_OF_TRUCKS = 0")

# Example usage
print("Truck Routes with Depot:")
print_schedule_for_each_day(truck_routes_extracted, delivery_days)


Truck Routes with Depot:

DAY = 2
NUMBER_OF_TRUCKS = 1
1 1 

DAY = 3
NUMBER_OF_TRUCKS = 1
1 2 

DAY = 4
NUMBER_OF_TRUCKS = 5
1 3 
2 4 
3 5 
4 6 
5 7 

DAY = 5
NUMBER_OF_TRUCKS = 2
1 8 
2 9 

DAY = 6
NUMBER_OF_TRUCKS = 1
1 10 


In [48]:
def print_schedule(truck_routes, delivery_days, assigned_requests, title, total_truck_distance, total_distance_all_techs):
    IDLE_MACHINE_COST = 0         # Define the actual cost
    
    # Calculate additional statistics
    days_with_trucks = set(day for _, day in truck_routes)
    days_with_technicians = set(day for day in delivery_days.values())
    all_days = sorted(set(days_with_trucks) | set(days_with_technicians))  # Union of days with trucks and technician days
    
    max_trucks_used = max(sum(1 for route, day in truck_routes if day == current_day) for current_day in days_with_trucks)
    max_technicians_used = max(sum(1 for day in delivery_days.values() if day == current_day) for current_day in days_with_technicians)
    
    number_of_truck_days = len(days_with_trucks)
    number_of_technician_days = len(days_with_technicians)

    # Calculate costs
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    # Print the additional information
    print("DATASET = ORTEC Caroline VeRoLog 2019")
    print(f"NAME = {title}")
    print(f"TRUCK_DISTANCE = {int(total_truck_distance)}")
    print(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}")
    print(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}")
    print(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}")
    print(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}")
    print(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}")
    print(f"IDLE_MACHINE_COSTS = {IDLE_MACHINE_COST}")
    print(f"TOTAL_COST = {int(total_cost)}")
    print()

    # Print the schedule for each day
    for current_day in all_days:
        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")
        
        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            print(truck_index, end=' ')
            for request_id in route:
                print(request_id, end=' ')  # Print each request ID in the route
            print()  # Newline after each truck route
        
        # Print technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        print(formatted_output)

# Example usage
print_schedule(truck_routes_extracted, delivery_days, assigned_requests, title, total_truck_distance, total_distance_all_techs)


DATASET = ORTEC Caroline VeRoLog 2019
NAME = Instance 5
TRUCK_DISTANCE = 4257
NUMBER_OF_TRUCK_DAYS = 5
NUMBER_OF_TRUCKS_USED = 5
TECHNICIAN_DISTANCE = 1700
NUMBER_OF_TECHNICIAN_DAYS = 5
NUMBER_OF_TECHNICIANS_USED = 5
IDLE_MACHINE_COSTS = 0
TOTAL_COST = 17000


DAY = 2
NUMBER_OF_TRUCKS = 1
1 1 
NUMBER_OF_TECHNICIANS = 0



DAY = 3
NUMBER_OF_TRUCKS = 1
1 2 
NUMBER_OF_TECHNICIANS = 1
3 1



DAY = 4
NUMBER_OF_TRUCKS = 5
1 3 
2 4 
3 5 
4 6 
5 7 
NUMBER_OF_TECHNICIANS = 1
1 2



DAY = 5
NUMBER_OF_TRUCKS = 2
1 8 
2 9 
NUMBER_OF_TECHNICIANS = 4
3 6
1 3 7
2 4
4 5



DAY = 6
NUMBER_OF_TRUCKS = 1
1 10 
NUMBER_OF_TECHNICIANS = 1
5 8 9




In [49]:
print(title)

Instance 5


In [50]:
def print_schedule(truck_routes, delivery_days, assigned_requests):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks
    
    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)
    
    all_days = sorted(all_days)  # Sort all unique days
    
    for current_day in all_days:
        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")
        
        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            print(truck_index, end=' ')
            for request_id in route:
                print(request_id, end=' ')  # Print each request ID in the route
            print()  # Newline after each truck route
        
        # Print technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        print(formatted_output)

# Example usage
print_schedule(truck_routes_extracted, delivery_days, assigned_requests)



DAY = 2
NUMBER_OF_TRUCKS = 1
1 1 
NUMBER_OF_TECHNICIANS = 0



DAY = 3
NUMBER_OF_TRUCKS = 1
1 2 
NUMBER_OF_TECHNICIANS = 1
3 1



DAY = 4
NUMBER_OF_TRUCKS = 5
1 3 
2 4 
3 5 
4 6 
5 7 
NUMBER_OF_TECHNICIANS = 1
1 2



DAY = 5
NUMBER_OF_TRUCKS = 2
1 8 
2 9 
NUMBER_OF_TECHNICIANS = 4
3 6
1 3 7
2 4
4 5



DAY = 6
NUMBER_OF_TRUCKS = 1
1 10 
NUMBER_OF_TECHNICIANS = 1
5 8 9



DAY = 7
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 1
3 10




In [51]:
def print_schedule(truck_routes, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks
    max_trucks_used = 0
    max_technicians_used = 0
    number_of_truck_days = 0
    number_of_technician_days = 0

    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)

    all_days = sorted(all_days)  # Sort all unique days

    for current_day in all_days:
        num_trucks_for_day = 0
        num_technicians_for_day = 0

        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")

        # Print truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            number_of_truck_days += 1
        if num_trucks_for_day > max_trucks_used:
            max_trucks_used = num_trucks_for_day
        print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            print(truck_index, end=' ')
            for request_id in route:
                print(request_id, end=' ')  # Print each request ID in the route
            print()  # Newline after each truck route

        # Print technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        num_technicians_for_day = formatted_output.count('NUMBER_OF_TECHNICIANS = ')
        if num_technicians_for_day > 0:
            technician_info = formatted_output.split('NUMBER_OF_TECHNICIANS = ')[1]
            num_technicians = int(technician_info.split()[0])
            if num_technicians > 0:
                number_of_technician_days += 1
                if num_technicians > max_technicians_used:
                    max_technicians_used = num_technicians
        if num_technicians_for_day > max_technicians_used:
            max_technicians_used = num_technicians_for_day
        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        print(formatted_output)

    
    IDLE_MACHINE_COST = total_idle_costs

    
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    return (number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost)


# Example usage
number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost = print_schedule(truck_routes_extracted, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs)

# Now you have access to these calculated values outside the function scope
print(f"TRUCK_DISTANCE = {int(total_truck_distance)}")
print(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}")
print(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}")
print(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}")
print(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}")
print(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}")
print(f"TOTAL_COST = {int(total_cost)}")



DAY = 2
NUMBER_OF_TRUCKS = 1
1 1 
NUMBER_OF_TECHNICIANS = 0



DAY = 3
NUMBER_OF_TRUCKS = 1
1 2 
NUMBER_OF_TECHNICIANS = 1
3 1



DAY = 4
NUMBER_OF_TRUCKS = 5
1 3 
2 4 
3 5 
4 6 
5 7 
NUMBER_OF_TECHNICIANS = 1
1 2



DAY = 5
NUMBER_OF_TRUCKS = 2
1 8 
2 9 
NUMBER_OF_TECHNICIANS = 4
3 6
1 3 7
2 4
4 5



DAY = 6
NUMBER_OF_TRUCKS = 1
1 10 
NUMBER_OF_TECHNICIANS = 1
5 8 9



DAY = 7
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 1
3 10


TRUCK_DISTANCE = 4257
NUMBER_OF_TRUCK_DAYS = 5
NUMBER_OF_TECHNICIAN_DAYS = 5
TECHNICIAN_DISTANCE = 1700
NUMBER_OF_TRUCKS_USED = 5
NUMBER_OF_TECHNICIANS_USED = 4
TOTAL_COST = 17000


In [54]:
def final_print_schedule(title, truck_routes, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks
    max_trucks_used = 0
    max_technicians_used = 0
    number_of_truck_days = 0
    number_of_technician_days = 0

    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)

    all_days = sorted(all_days)  # Sort all unique days

    # Ensure DAY = 1 is included even if it's not present in the data
    if 1 not in all_days:
        all_days.insert(0, 1)

    # Initialize formatted day-by-day output
    daily_output = []

    for current_day in all_days:
        num_trucks_for_day = 0
        num_technicians_for_day = 0

        day_output = []
        day_output.append(f"DAY = {current_day}")

        # Check if there are any truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            number_of_truck_days += 1
        if num_trucks_for_day > max_trucks_used:
            max_trucks_used = num_trucks_for_day
        day_output.append(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            route_output = f"{truck_index} " + ' '.join(map(str, route))
            day_output.append(route_output)

        # Check if there are any technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        num_technicians_for_day = formatted_output.count('NUMBER_OF_TECHNICIANS = ')
        if num_technicians_for_day > 0:
            technician_info = formatted_output.split('NUMBER_OF_TECHNICIANS = ')[1]
            num_technicians = int(technician_info.split()[0])
            if num_technicians > 0:
                number_of_technician_days += 1
                if num_technicians > max_technicians_used:
                    max_technicians_used = num_technicians
        if num_technicians_for_day > max_technicians_used:
            max_technicians_used = num_technicians_for_day

        # Check if both truck routes and technician assignments are empty for the current day
        if num_trucks_for_day == 0 and num_technicians_for_day == 0:
            day_output.append("NUMBER_OF_TRUCKS = 0")
            day_output.append("NUMBER_OF_TECHNICIANS = 0")

        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        day_output.append(formatted_output)

        daily_output.append('\n'.join(day_output))

    # Print summary information
    IDLE_MACHINE_COST = total_idle_costs
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    print(f"DATASET = ORTEC Caroline VeRoLog 2019")
    print(f"NAME = {title}")
    print(f"TRUCK_DISTANCE = {int(total_truck_distance)}")
    print(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}")
    print(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}")
    print(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}")
    print(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}")
    print(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}")
    print(f"IDLE_MACHINE_COSTS = {total_idle_costs}")
    print(f"TOTAL_COST = {int(total_cost)}")

    # Print the day-by-day output
    for output in daily_output:
        print()
        print(output)

    return (number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost)


number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost = final_print_schedule(title, truck_routes_extracted, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs)


DATASET = ORTEC Caroline VeRoLog 2019
NAME = Instance 5
TRUCK_DISTANCE = 4257
NUMBER_OF_TRUCK_DAYS = 5
NUMBER_OF_TECHNICIAN_DAYS = 5
TECHNICIAN_DISTANCE = 1700
NUMBER_OF_TRUCKS_USED = 5
NUMBER_OF_TECHNICIANS_USED = 4
IDLE_MACHINE_COSTS = 0
TOTAL_COST = 17000

DAY = 1
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 0



DAY = 2
NUMBER_OF_TRUCKS = 1
1 1
NUMBER_OF_TECHNICIANS = 0



DAY = 3
NUMBER_OF_TRUCKS = 1
1 2
NUMBER_OF_TECHNICIANS = 1
3 1



DAY = 4
NUMBER_OF_TRUCKS = 5
1 3
2 4
3 5
4 6
5 7
NUMBER_OF_TECHNICIANS = 1
1 2



DAY = 5
NUMBER_OF_TRUCKS = 2
1 8
2 9
NUMBER_OF_TECHNICIANS = 4
3 6
1 3 7
2 4
4 5



DAY = 6
NUMBER_OF_TRUCKS = 1
1 10
NUMBER_OF_TECHNICIANS = 1
5 8 9



DAY = 7
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 1
3 10




In [53]:
def final_print_schedule(title, truck_routes, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs, output_file_name):
    all_days = set(delivery_days.values())  # Get all unique delivery days from trucks
    max_trucks_used = 0
    max_technicians_used = 0
    number_of_truck_days = 0
    number_of_technician_days = 0

    # Get all unique days from technician assignments
    for requests in assigned_requests.values():
        for _, day in requests:
            all_days.add(day)

    all_days = sorted(all_days)  # Sort all unique days

    # Initialize formatted day-by-day output
    daily_output = []

    # Ensure DAY = 1 is included even if it's not present in the data
    if 1 not in all_days:
        all_days.insert(0, 1)

    for current_day in all_days:
        num_trucks_for_day = 0
        num_technicians_for_day = 0

        day_output = []
        day_output.append(f"DAY = {current_day}")

        # Check if there are any truck routes for the current day
        truck_routes_for_day = [route for route, day in truck_routes if day == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            number_of_truck_days += 1
        if num_trucks_for_day > max_trucks_used:
            max_trucks_used = num_trucks_for_day
        day_output.append(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            route_output = f"{truck_index} " + ' '.join(map(str, route))
            day_output.append(route_output)

        # Check if there are any technician assignments for the current day
        formatted_output = format_assigned_requests(assigned_requests, current_day)
        num_technicians_for_day = formatted_output.count('NUMBER_OF_TECHNICIANS = ')
        if num_technicians_for_day > 0:
            technician_info = formatted_output.split('NUMBER_OF_TECHNICIANS = ')[1]
            num_technicians = int(technician_info.split()[0])
            if num_technicians > 0:
                number_of_technician_days += 1
                if num_technicians > max_technicians_used:
                    max_technicians_used = num_technicians
        if num_technicians_for_day > max_technicians_used:
            max_technicians_used = num_technicians_for_day

        # Check if both truck routes and technician assignments are empty for the current day
        if num_trucks_for_day == 0 and num_technicians_for_day == 0:
            day_output.append("NUMBER_OF_TRUCKS = 0")
            day_output.append("NUMBER_OF_TECHNICIANS = 0")

        # Remove the DAY line from the formatted output
        formatted_output = '\n'.join(formatted_output.split('\n')[1:])
        day_output.append(formatted_output)

        daily_output.append('\n'.join(day_output))

    # Calculate costs
    IDLE_MACHINE_COST = total_idle_costs
    total_truck_distance_cost = total_truck_distance * TRUCK_DISTANCE_COST
    total_truck_day_cost = number_of_truck_days * TRUCK_DAY_COST
    total_truck_cost = max_trucks_used * TRUCK_COST
    total_technician_distance_cost = total_distance_all_techs * TECHNICIAN_DISTANCE_COST
    total_technician_day_cost = number_of_technician_days * TECHNICIAN_DAY_COST
    total_technician_cost = max_technicians_used * TECHNICIAN_COST

    total_cost = (total_truck_distance_cost + total_truck_day_cost + total_truck_cost +
                  total_technician_distance_cost + total_technician_day_cost + total_technician_cost + IDLE_MACHINE_COST)

    # Open a file to write the output
    with open(output_file_name, "w") as file:
        # Write the summary information
        file.write(f"DATASET = ORTEC Caroline VeRoLog 2019\n")
        file.write(f"NAME = {title}\n")
        file.write(f"TRUCK_DISTANCE = {int(total_truck_distance)}\n")
        file.write(f"NUMBER_OF_TRUCK_DAYS = {number_of_truck_days}\n")
        file.write(f"NUMBER_OF_TECHNICIAN_DAYS = {number_of_technician_days}\n")
        file.write(f"TECHNICIAN_DISTANCE = {int(total_distance_all_techs)}\n")
        file.write(f"NUMBER_OF_TRUCKS_USED = {max_trucks_used}\n")
        file.write(f"NUMBER_OF_TECHNICIANS_USED = {max_technicians_used}\n")
        file.write(f"IDLE_MACHINE_COSTS = {total_idle_costs}\n")
        file.write(f"TOTAL_COST = {int(total_cost)}\n")

        # Write the day-by-day output
        for output in daily_output:
            file.write("\n")
            file.write(output)
            file.write("\n")

    return (number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost)


number_of_truck_days, number_of_technician_days, max_trucks_used, max_technicians_used, total_cost = final_print_schedule(title, truck_routes_extracted, delivery_days, assigned_requests, total_truck_distance, total_distance_all_techs, output_file_name)
